In [1]:
# # for google colab
# from kaggle_datasets import KaggleDatasets
# KaggleDatasets().get_gcs_path()

In [2]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 1.5 MB/s eta 0:00:0000:0100:01


In [3]:
import os
import gc
import zipfile
import torch

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import cv2
# from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torchvision import models
import torchvision
import torch.nn as nn
import timm
from torchvision import transforms
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
args = {}
model_name = 'tf_efficientnetv2_m'
data_config = timm.data.resolve_data_config({}, model=model_name, verbose=True)
print(data_config["mean"], data_config["std"])

(0.485, 0.456, 0.406) (0.229, 0.224, 0.225)


In [6]:
N = 16

In [7]:
n_class = 2
backbone = timm.create_model(model_name, pretrained=True, num_classes=0).to(device)
model = nn.Sequential(
    backbone,
    nn.Dropout(0.2),
    nn.Linear(backbone.num_features, n_class)
).to(device)
# params_to_update = []
# update_param_names = ["effnet.classifier.weight", "effnet.classifier.bias"]
# for name, param in model.named_parameters():
#     if name in update_param_names:
#         param.requires_grad = True
#         params_to_update.append(param)
#     else:
#         param.requires_grad = False

In [8]:
train = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
sum(train["image_id"] == "2c3c06_0")

0

In [9]:
# This block is reffered from https://www.kaggle.com/code/yasufuminakama/mayo-train-images-size-1024-n-16-1/notebook
train = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
# train = train[train["image_id"] != "2c3c06_0"]

train['image_dir'] = ''
# train.loc[:100,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_1/'
# train.loc[100:200,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_2/'
# train.loc[200:300,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_3/'
# train.loc[300:400,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_4/'
# train.loc[400:500,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_5/'
# train.loc[500:600,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_6/'
# train.loc[600:700,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_7/'
# train.loc[700:,'image_dir'] = '/kaggle/input/mayo-train-images-size1024-n16/train_images_8/'

train.loc[:100,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_1/'
train.loc[100:200,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_2/'
train.loc[200:300,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_3/'
train.loc[300:400,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_4/'
train.loc[400:500,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_5/'
train.loc[500:600,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_6/'
train.loc[600:700,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_7/'
train.loc[700:,'image_dir'] = '/kaggle/input/mayo-tiled-16-384x384/train_images/train_images/train_images_8/'


target_mapper = {"CE": 0, "LAA": 1}

train["target"] = train["label"].map(lambda x: target_mapper[x])

train_full = pd.DataFrame()
for i in range(N):
    train["path"] = train["image_dir"] + train["image_id"] + f'_{i}.jpg'
    if i == 0:
        train_full = train
    else:
        train_full = pd.concat([train_full, train], axis=0)

train_full.reset_index(inplace=True)

class TrainDataset(Dataset):
    def __init__(self, cfg, df, transform=None, aug=True):
        self.cfg = cfg
        self.image_ids = df['image_id'].values
        self.image_dirs = df['image_dir'].values
        self.image_path = df["path"].values
        self.labels = df['target'].values
        self.transform = transform
        self.aug = aug

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
#         image_id = self.image_ids[idx]
#         image_dir = self.image_dirs[idx]
#         images = []
#         for i in range(16):
#             path = image_dir + image_id + f'_{i}.jpg'
#             image = cv2.imread(path)
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             image = self.transform(image)
# #             image = self.transform(image=image)["image"]            
#             images.append(image)
#         images = torch.stack(images, dim=0)
        image = cv2.imread(self.image_path[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = image.transpose((2, 0, 1)) 
        image = transforms.ToPILImage()(image)
        if self.aug:
            image = torchvision.transforms.RandAugment()(image)
        image = self.transform(image)
        label = torch.tensor(self.labels[idx]).long()
        return image, label

In [10]:
# transform = transforms.Compose([
#                     transforms.ToTensor(),
#                     transforms.Normalize([0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
#                     ])
config = resolve_data_config({}, model=backbone)
transform = create_transform(**config)
transform

Compose(
    Resize(size=384, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(384, 384))
    ToTensor()
    Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
)

In [11]:
for i in train_full["path"]:
    if not os.path.exists(i):
        print(i+ " does not exist!")

In [12]:
training_set = TrainDataset(None,df=train_full, transform=transform, aug=False)
training_loader = torch.utils.data.DataLoader(training_set, shuffle=True, num_workers=2, batch_size=16)

In [13]:
loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [14]:
model.parameters()

<generator object Module.parameters at 0x7f0e0a489cd0>

In [15]:
def train_one_epoch(model, device, train_loader, optimizer, epoch, loss_fn):
    running_loss = 0.
    last_loss = 0.
    model.train(True)

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, (data, target) in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data.to(device), target.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 100 == 99:
            last_loss = running_loss / 100 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch * len(train_loader) + i + 1
            running_loss = 0.

    return last_loss

In [16]:
# print(model)

In [17]:
# Initializing in a separate cell so we can easily add more epochs to the same run
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    avg_loss = train_one_epoch(model=model, device=device, train_loader=training_loader, optimizer=optimizer, epoch=epoch, loss_fn=loss_fn)

    epoch_number += 1

EPOCH 1:
  batch 100 loss: 0.6090686586499214
  batch 200 loss: 0.6175787782669068
  batch 300 loss: 0.5898023504018783
  batch 400 loss: 0.596968335211277
  batch 500 loss: 0.6055354446172714
  batch 600 loss: 0.6055685725808143
  batch 700 loss: 0.6005667978525162
EPOCH 2:
  batch 100 loss: 0.5871924465894699
  batch 200 loss: 0.6070717304944993
  batch 300 loss: 0.5716262599825859
  batch 400 loss: 0.5911877915263176
  batch 500 loss: 0.578797998726368
  batch 600 loss: 0.5926144686341286
  batch 700 loss: 0.5971759322285652
EPOCH 3:
  batch 100 loss: 0.5910799673199654
  batch 200 loss: 0.5946392294764519
  batch 300 loss: 0.589988742172718
  batch 400 loss: 0.5963460010290146
  batch 500 loss: 0.5879538995027542
  batch 600 loss: 0.577158868610859
  batch 700 loss: 0.5860155895352364
EPOCH 4:
  batch 100 loss: 0.5942493346333504
  batch 200 loss: 0.585308445096016
  batch 300 loss: 0.5841214454174042
  batch 400 loss: 0.5682286441326141
  batch 500 loss: 0.6016463208198547
  batch

In [18]:
torch.save(model.state_dict(), "best_model_384.pth")